In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models , transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import torchvision
import numpy as np
import time


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)



Using device: cuda


In [7]:
image_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2,contrast=0.2),
    transforms.RandomRotation(10),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225))]    
)

In [8]:
dataset_path = dataset_path = r"C:\Data_Learnings\Internship2\FreshHarvest_Dataset\FRUIT-16K"
dataset = datasets.ImageFolder(dataset_path,transform=image_transform)
len(dataset)

16000

In [9]:
dataset.classes

['F_Banana',
 'F_Lemon',
 'F_Lulo',
 'F_Mango',
 'F_Orange',
 'F_Strawberry',
 'F_Tamarillo',
 'F_Tomato',
 'S_Banana',
 'S_Lemon',
 'S_Lulo',
 'S_Mango',
 'S_Orange',
 'S_Strawberry',
 'S_Tamarillo',
 'S_Tomato']

In [10]:
num_classes = len(dataset.classes)
num_classes

16

In [11]:
train_size = int(0.8 * len(dataset))
train_size

12800

In [12]:
val_size = int(len(dataset) - train_size)
val_size

3200

In [13]:
from torch.utils.data import random_split
train_dataset, val_dataset = random_split(dataset, [train_size, val_size]) # splitting the image dataset into training and validataion dataset

In [14]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True) # saving the images in train_loader
val_loader = DataLoader(val_dataset,batch_size=32,shuffle=True) 

In [15]:
for images,labels in train_loader:
    print(images.shape) # torch.Size([32, 3, 224, 224]) → batch of 32 images, each with 3 color channels and size 224x224
    print(labels.shape)  # torch.Size([32]) → batch of 32 labels
    break

torch.Size([32, 3, 224, 224])
torch.Size([32])


In [16]:
images[31],labels[31]


(tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],
 
         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],
 
         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044, -1.8044,

In [17]:
class FruitClassifier(nn.Module):
    def __init__(self,num_classes):
        super(FruitClassifier,self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3,16,kernel_size=3,stride = 1,padding=1), # (16,224,224) #Without padding, image size shrinks.
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2,padding=0), # (16,112,112) #stride=2 means we are taking 2 steps at a time, so the image size is reduced by half.
            nn.Conv2d(16,32,kernel_size=3,stride = 1,padding=1), # (32,112,112) #Without padding, image size shrinks.
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2,padding=0), # (32,56,56)
            nn.Conv2d(32,64,kernel_size=3,stride = 1,padding=1), # (64,56,56) #Without padding, image size shrinks.
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2,padding=0), # (64,28,28)
            nn.Flatten(),
            nn.Linear(64*28*28,512),
            nn.ReLU(),
            nn.Linear(512,num_classes)
        )



    def forward(self,x):
         x = self.network(x)  
         return x     

In [18]:
model = FruitClassifier(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [21]:
def train(model,train_loader,val_loader,criterion,optimizer,num_epochs):
    start = time.time()
    model.train() # set the model to training mode
    running_loss = 0.0
    for epoch in range(num_epochs):
        for batch_num,(images,labels) in enumerate(train_loader):
            images,labels = images.to(device),labels.to(device)

            # zero the gradient parameter 
            optimizer.zero_grad()
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs,labels)
            # Backward pass and optimization
            loss.backward() # AVERAGE LOSS FOR BATCH OF 32 IMAGES
            optimizer.step()
            if (batch_num+1) % 10 == 0: # print loss every 10 batches
                print(f"Epoch {epoch+1}, Batch {batch_num+1}, Loss: {loss.item():.4f}")
            running_loss += loss.item() * images.size(0) # TOTAL LOSS FOR THE BATCH OF 32 IMAGES
    epoch_loss = running_loss / len(train_loader.dataset) # AVERAGE LOSS PER IMAGE
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
    ## validation
    model.eval() # set the model to evaluation mode
    correct = 0
    total = 0       
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for images,labels in val_loader:
            images,labels = images.to(device),labels.to(device)
            outputs = model(images)
            _,predicted = torch.max(outputs.data,1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy()) # GPU tensors cannot be converted directly to NumPy
            all_predictions.extend(predicted.cpu().numpy())
        print(f"Validation Accuracy: {100 * correct / total:.2f}%")
    end = time.time()
    print(f"Training and validation completed in {end - start:.2f} seconds.")
    return all_labels,all_predictions  

In [23]:
train(model,train_loader,val_loader,criterion,optimizer,num_epochs=2)

Epoch 1, Batch 10, Loss: 0.5206
Epoch 1, Batch 20, Loss: 0.6175
Epoch 1, Batch 30, Loss: 0.5180
Epoch 1, Batch 40, Loss: 0.6820
Epoch 1, Batch 50, Loss: 0.5472
Epoch 1, Batch 60, Loss: 0.6170
Epoch 1, Batch 70, Loss: 0.7893
Epoch 1, Batch 80, Loss: 0.5192
Epoch 1, Batch 90, Loss: 0.4205
Epoch 1, Batch 100, Loss: 0.5646
Epoch 1, Batch 110, Loss: 0.3454
Epoch 1, Batch 120, Loss: 0.7298
Epoch 1, Batch 130, Loss: 0.3601
Epoch 1, Batch 140, Loss: 0.7242
Epoch 1, Batch 150, Loss: 0.5070
Epoch 1, Batch 160, Loss: 0.4931
Epoch 1, Batch 170, Loss: 0.3471
Epoch 1, Batch 180, Loss: 0.4264
Epoch 1, Batch 190, Loss: 0.4187
Epoch 1, Batch 200, Loss: 0.5447
Epoch 1, Batch 210, Loss: 0.2992
Epoch 1, Batch 220, Loss: 0.7579
Epoch 1, Batch 230, Loss: 0.6866
Epoch 1, Batch 240, Loss: 0.5023
Epoch 1, Batch 250, Loss: 0.4132
Epoch 1, Batch 260, Loss: 0.6590
Epoch 1, Batch 270, Loss: 0.4413
Epoch 1, Batch 280, Loss: 0.4033
Epoch 1, Batch 290, Loss: 0.7197
Epoch 1, Batch 300, Loss: 0.5896
Epoch 1, Batch 310,

([np.int64(9),
  np.int64(13),
  np.int64(10),
  np.int64(2),
  np.int64(0),
  np.int64(1),
  np.int64(7),
  np.int64(1),
  np.int64(6),
  np.int64(5),
  np.int64(0),
  np.int64(0),
  np.int64(2),
  np.int64(6),
  np.int64(11),
  np.int64(8),
  np.int64(8),
  np.int64(7),
  np.int64(12),
  np.int64(11),
  np.int64(9),
  np.int64(9),
  np.int64(10),
  np.int64(9),
  np.int64(0),
  np.int64(11),
  np.int64(9),
  np.int64(7),
  np.int64(12),
  np.int64(1),
  np.int64(1),
  np.int64(11),
  np.int64(11),
  np.int64(1),
  np.int64(5),
  np.int64(6),
  np.int64(1),
  np.int64(13),
  np.int64(3),
  np.int64(12),
  np.int64(3),
  np.int64(8),
  np.int64(11),
  np.int64(5),
  np.int64(15),
  np.int64(9),
  np.int64(11),
  np.int64(13),
  np.int64(11),
  np.int64(0),
  np.int64(6),
  np.int64(12),
  np.int64(14),
  np.int64(0),
  np.int64(5),
  np.int64(4),
  np.int64(12),
  np.int64(3),
  np.int64(0),
  np.int64(11),
  np.int64(13),
  np.int64(0),
  np.int64(10),
  np.int64(15),
  np.int64(0),
 